# Empirical Example

### Introduction: Mammal phylogeny
The purpose of this notebook is to demonstrate `ipcoal` simulations on a topology inferred from empirical data, along with empirical data on generation times and species distribution areas (as a proxy for effective population sizes).  
  
`ipcoal` results can show us the amount of discordance to expect from one topology to the next in this scenario, and the expected lengths of genealogies.

**Imports:**

In [1]:
import numpy as np
import toytree
import pandas as pd
import toyplot
import ipcoal
from copy import deepcopy
colormap = toyplot.color.brewer.map("BlueRed", reverse=True)

**Data sources:**

1) Species tree: dated maximum clade credibility tree from a recent mammal supertree analysis (Upham et al. 2019: https://doi.org/10.1371/journal.pbio.3000494)  
2) Species distribution areas: from PanTHERIA database (Jones et al 2009: https://doi.org/10.1890/08-1494.1)  
3) Species generation lengths: from generation lengths dataset (Pacifici et al. 2014: https://doi.org/10.5061/dryad.gd0m3)

### Consolidate data

Pull together data from three data sources: species tree, species distribution areas, and species generation lengths, and align the data to eliminate species that aren't shared among the three datasets.

**Names from species tree:**

In [2]:
tree = toytree.tree("mammal_dat/MamPhy_fullPosterior_BDvr_DNAonly_4098sp_topoFree_NDexp_MCC_v2_target.tre",
                       tree_format=10)

In [3]:
trenames = np.array(["_".join(i.split("_")[:2]) for i in tree.get_tip_labels()])

**Names from PanTHERIA data:**

In [4]:
panth = pd.read_csv('./mammal_dat/PanTHERIA_1-0_WR05_Aug2008.txt',sep='\t')

In [5]:
panth.head()

,MSW05_Order,MSW05_Family,MSW05_Genus,MSW05_Species,MSW05_Binomial,1-1_ActivityCycle,5-1_AdultBodyMass_g,8-1_AdultForearmLen_mm,13-1_AdultHeadBodyLen_mm,2-1_AgeatEyeOpening_d,...,26-6_GR_MinLong_dd,26-7_GR_MidRangeLong_dd,27-1_HuPopDen_Min_n/km2,27-2_HuPopDen_Mean_n/km2,27-3_HuPopDen_5p_n/km2,27-4_HuPopDen_Change,28-1_Precip_Mean_mm,28-2_Temp_Mean_01degC,30-1_AET_Mean_mm,30-2_PET_Mean_mm
0,Artiodactyla,Camelidae,Camelus,dromedarius,Camelus dromedarius,3.0,492714.47,-999.0,-999.00,-999.00,...,-999.00,-999.00,-999,-999.00,-999.0,-999.00,-999.00,-999.00,-999.00,-999.00
1,Carnivora,Canidae,Canis,adustus,Canis adustus,1.0,10392.49,-999.0,745.32,-999.00,...,-17.53,13.00,0,35.20,1.0,0.14,90.75,236.51,922.90,1534.40
2,Carnivora,Canidae,Canis,aureus,Canis aureus,2.0,9658.70,-999.0,827.53,7.50,...,-17.05,45.74,0,79.29,0.0,0.10,44.61,217.23,438.02,1358.98
3,Carnivora,Canidae,Canis,latrans,Canis latrans,2.0,11989.10,-999.0,872.39,11.94,...,-168.12,-117.60,0,27.27,0.0,0.06,53.03,58.18,503.02,728.37
4,Carnivora,Canidae,Canis,lupus,Canis lupus,2.0,31756.51,-999.0,1055.00,14.01,...,-171.84,3.90,0,37.87,0.0,0.04,34.79,4.82,313.33,561.11


In [6]:
panthnames = [i.split(" ") for i in panth['MSW05_Binomial']]

In [7]:
panthnames = np.array(["_".join(i) for i in panthnames])

**Names from Nature Cons data:**

In [8]:
natcons = pd.read_csv('./mammal_dat/5734-SP-2-Editor.csv',sep=',')

In [9]:
natcons.head()

,TaxID,Order,Family,Genus,Scientific_name,AdultBodyMass_g,Sources_AdultBodyMass,Max_longevity_d,Sources_Max_longevity,Rspan_d,AFR_d,Data_AFR,Calculated_GL_d,GenerationLength_d,Sources_GL
0,42641,Rodentia,Muridae,Abditomys,Abditomys latidens,268.09,PanTHERIA,no information,no information,no information,no information,no information,no information,639.631832,Mean_family_same_body_mass
1,17879,Rodentia,Muridae,Abeomelomys,Abeomelomys sevia,54.88,PanTHERIA,no information,no information,no information,no information,no information,no information,624.399641,Mean_family_same_body_mass
2,16,Rodentia,Cricetidae,Abrawayaomys,Abrawayaomys ruschii,62.99,PanTHERIA,no information,no information,no information,no information,no information,no information,589.388299,Mean_family_same_body_mass
3,42656,Rodentia,Abrocomidae,Abrocoma,Abrocoma bennettii,250.50,PanTHERIA,839.5,PanTHERIA;AnAge,no information,no information,no information,no information,1032.923574,Mean_order_same_mass
4,18,Rodentia,Abrocomidae,Abrocoma,Abrocoma boliviensis,158.00,PanTHERIA,no information,no information,no information,no information,no information,no information,1032.923574,Mean_order_same_mass


In [10]:
natconsnames = np.array(["_".join(q) for q in [i.split(" ") for i in natcons['Scientific_name']]])

**See which names are common to all lists:**

In [11]:
# append name to intersection list if it is in all datasets
intersection_names = []
for trename in trenames:
    if trename in panthnames:
        if trename in natconsnames:
            intersection_names.append(trename)

In [12]:
# how many names are in all three datasets?
len(intersection_names)

3489

In [13]:
# start a dataframe to build up
intersect_df = pd.DataFrame({'scientific_name':intersection_names})

In [14]:
# look at dataframe -- currently just rows of scientific names
intersect_df.head()

,scientific_name
0,Akodon_boliviensis
1,Akodon_spegazzinii
2,Akodon_sylvanus
3,Akodon_lutescens
4,Akodon_subfuscus


**Gather the data and write to csv:**

In [16]:
gl_dat = np.zeros((len(intersect_df),1))
sp_area_dat = np.zeros((len(intersect_df),1))

for idx, name in enumerate(intersect_df['scientific_name']):
    # using mean because some entries have multiple generation lengths (from different datasets)
    gl_dat[idx] = np.mean(natcons['GenerationLength_d'][natcons['Scientific_name'] == " ".join(name.split("_"))])
    sp_area_dat[idx] = np.mean(panth['26-1_GR_Area_km2'][panth['MSW05_Binomial'] == " ".join(name.split("_"))])

In [17]:
# add data columns to the dataframe
intersect_df['generation_length'] = gl_dat
intersect_df['species_area'] = sp_area_dat

In [18]:
# look at the first few rows
intersect_df.head()

,scientific_name,generation_length,species_area
0,Akodon_boliviensis,589.388299,530195.66
1,Akodon_spegazzinii,589.388299,83477.58
2,Akodon_sylvanus,589.388299,48393.63
3,Akodon_lutescens,589.388299,318909.65
4,Akodon_subfuscus,589.388299,353423.95


In [19]:
# some of the data lists NaN vals with -999. Let's remove these.
intersect_df = intersect_df[~intersect_df.species_area.eq(-999)]

**Trim the species tree down to just the tips shared among datasets, and write that out as a new file:**

In [23]:
tips = tree.get_tip_labels()

In [24]:
dropped_tips = np.array(tips)[~np.array([q in np.array(intersect_df.scientific_name) for 
                                         q in ["_".join(i.split("_")[0:2]) for i in tips]])]

In [25]:
tree = tree.drop_tips(list(dropped_tips))

### Merge the species data with the phylogeny

We can see that the number of tree labels is that same as the number of rows in the dataframe:

In [29]:
len(tree.get_tip_labels())

3121

In [30]:
intersect_df.shape[0]

3121

Designate minimum and maximum values for Ne:

In [31]:
max_Ne = 150000
min_Ne = 50000
Ne_range = max_Ne - min_Ne

In [32]:
max_area = np.max(intersect_df.species_area)
min_area = np.min(intersect_df.species_area)
area_range = max_area - min_area

Add an `Ne` column where we've scaled species area to fit between a minimum and maximum Ne value:

In [33]:
intersect_df['Ne'] = ( ((np.array(intersect_df.species_area)-min_area) / area_range) * Ne_range ) + min_Ne

In [34]:
intersect_df.head()

,scientific_name,generation_length,species_area,Ne
0,Akodon_boliviensis,589.388299,530195.66,50841.122410
1,Akodon_spegazzinii,589.388299,83477.58,50132.431984
2,Akodon_sylvanus,589.388299,48393.63,50076.773481
3,Akodon_lutescens,589.388299,318909.65,50505.930308
4,Akodon_subfuscus,589.388299,353423.95,50560.685096


**Set tip vals on species tree for Ne and generation length:**

In [35]:
# first make a column where the literal tip label is matched to species name
intersect_df["tip_label"] = np.array(tree.get_tip_labels())[np.array([np.argmax(np.array(["_".join(i.split("_")[:2]) for i in tree.get_tip_labels()]) == q) for q in intersect_df.scientific_name])]
intersect_df.head()

,scientific_name,generation_length,species_area,Ne,tip_label
0,Akodon_boliviensis,589.388299,530195.66,50841.122410,Akodon_boliviensis_CRICE...
1,Akodon_spegazzinii,589.388299,83477.58,50132.431984,Akodon_spegazzinii_CRICE...
2,Akodon_sylvanus,589.388299,48393.63,50076.773481,Akodon_sylvanus_CRICETID...
3,Akodon_lutescens,589.388299,318909.65,50505.930308,Akodon_lutescens_CRICETI...
4,Akodon_subfuscus,589.388299,353423.95,50560.685096,Akodon_subfuscus_CRICETI...


In [36]:
# match up Ne and g to the tip labels
ne_dict = dict(zip(np.array(intersect_df.tip_label),np.array(intersect_df.Ne)))
g_dict = dict(zip(np.array(intersect_df.tip_label),np.array(intersect_df.generation_length)))

In [37]:
# set the tip labels for both Ne and g
tree = tree.set_node_values("g",g_dict)
tree = tree.set_node_values("Ne",ne_dict)

**Ancestral state reconstruction:**  
  
This is done using `toytree`'s PCM module.

In [38]:
recon_tree = toytree.PCM.PCM(tree)
ntree = recon_tree.ancestral_state_reconstruction("g")

In [39]:
recon_tree = toytree.PCM.PCM(ntree)
ntree = recon_tree.ancestral_state_reconstruction("Ne")

### Simulate on a subset of the species tree  
  
We'll pick out a genus from Carnivora.

In [40]:
# copy the tree to prune it down
s_plot = deepcopy(ntree)

In [41]:
# select the genus Mustela
mrca = s_plot.get_mrca_idx_from_tip_labels(wildcard="Mustela_")
snode = s_plot.treenode.search_nodes(idx=mrca)[0]
s_plot.treenode.prune(snode.get_descendants())
s_plot.ntips

15

In [42]:
# scale the height from millions of years to years
s_plot = s_plot.mod.node_scale_root_height(s_plot.treenode.height*1e6)
s_plot.treenode.height

8077930.0

**Visualize the species tree with generation time (node colors) and Ne (edge widths) mappled on:**

In [43]:
# draw tree with reconstructed g values as node colors and Ne values as edge widths
s_plot.draw(
    ts='p', 
    #node_labels=recon_tree.get_node_values("Ne", 1, 1),
    node_colors=[colormap.colors(i, 
                                 np.min(s_plot.get_node_values("g",1,1)), 
                                 np.max(s_plot.get_node_values("g",1,1))
                                ) for i in s_plot.get_node_values('g', 1, 1)],
    height=500,
    width=600,
);

<svg class="toyplot-canvas-Canvas" height="500.0px" id="t5ee442af1b164f399a32dded59096fac" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600.0 500.0" width="600.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Mustela_eversmanii_MUSTELIDAE_CARNIVORA Mustela_putorius_MUSTELIDAE_CARNIVORA Mustela_lutreola_MUSTELIDAE_CARNIVORA Mustela_sibirica_MUSTELIDAE_CARNIVORA Mustela_itatsi_MUSTELIDAE_CARNIVORA Mustela_altaica_MUSTELIDAE_CARNIVORA Mustela_nivalis_MUSTELIDAE_CARNIVORA Mustela_erminea_MUSTELIDAE_CARNIVORA Mustela_kathiah_MUSTELIDAE_CARNIVORA Mustela_nudipes_MUSTELIDAE_CARNIVORA Mustela_strigidorsa_MUSTELIDAE_CARNIVORA Mustela_felipei_MUSTELIDAE_CARNIVORA Mustela_africana_MUSTELIDAE_CARNIVORA Mustela_frenata_MUSTELIDAE_CARNIVORA Neovison_vison_MUSTELIDAE_CARNIVORA idx: 0
name: Mustela_eversmanii_MUSTELIDAE_CARNIVORA
dist: 533987.0000
support: 0
height: 6.0000
g: 1222.3850
Ne: 76262.1310 0 idx: 1
name: Mustela_putorius_MUSTELIDAE_CARNIVORA
dist: 533987.0000
support: 0
height: 6.0000
g: 1652.4644
Ne: 62002.2603 1 idx: 2
name: Mustela_lutreola_MUSTELIDAE_CARNIVORA
dist: 1343270.0000
support: 0
height: 8.0000
g: 730.0000
Ne: 55661.9256 2 idx: 3
name: Mustela_sibirica_MUSTELIDAE_CARNIVORA
dist: 1702970.0000
support: 0
height: 10.0000
g: 1484.7922
Ne: 82013.4602 3 idx: 4
name: Mustela_itatsi_MUSTELIDAE_CARNIVORA
dist: 2295590.0000
support: 0
height: 6.0000
g: 1931.4500
Ne: 50574.9310 4 idx: 5
name: Mustela_altaica_MUSTELIDAE_CARNIVORA
dist: 2760080.0000
support: 0
height: 14.0000
g: 1846.9000
Ne: 62140.0346 5 idx: 6
name: Mustela_nivalis_MUSTELIDAE_CARNIVORA
dist: 2760080.0000
support: 0
height: 14.0000
g: 1194.1940
Ne: 119258.9241 6 idx: 7
name: Mustela_erminea_MUSTELIDAE_CARNIVORA
dist: 4368490.0000
support: 0
height: 8.0000
g: 1078.7779
Ne: 113010.2376 7 idx: 8
name: Mustela_kathiah_MUSTELIDAE_CARNIVORA
dist: 6105860.0000
support: 0
height: 8.0000
g: 1253.0400
Ne: 50508.5112 8 idx: 9
name: Mustela_nudipes_MUSTELIDAE_CARNIVORA
dist: 4460910.0000
support: 0
height: 8.0000
g: 2407.9050
Ne: 52269.5290 9 idx: 10
name: Mustela_strigidorsa_MUSTELIDAE_CARNIVORA
dist: 4460910.0000
support: 0
height: 8.0000
g: 2333.8100
Ne: 50719.7194 10 idx: 11
name: Mustela_felipei_MUSTELIDAE_CARNIVORA
dist: 2268980.0000
support: 0
height: 10.0000
g: 1939.3683
Ne: 50003.5834 11 idx: 12
name: Mustela_africana_MUSTELIDAE_CARNIVORA
dist: 2268980.0000
support: 0
height: 10.0000
g: 1572.6600
Ne: 56899.4500 12 idx: 13
name: Mustela_frenata_MUSTELIDAE_CARNIVORA
dist: 2785060.0000
support: 0
height: 0.0000
g: 986.1105
Ne: 69498.7776 13 idx: 14
name: Neovison_vison_MUSTELIDAE_CARNIVORA
dist: 5306730.0000
support: 0
height: 10.0000
g: 1473.6500
Ne: 70970.4741 14 idx: 15
name: 3350
dist: 809285.0000
support: 0
height: 533993.0000
g: 1437.4247
Ne: 69132.1956 15 idx: 16
name: 3426
dist: 359702.0000
support: 0
height: 1343278.0000
g: 1122.7437
Ne: 63140.2669 16 idx: 17
name: 3526
dist: 592616.0000
support: 0
height: 1702980.0000
g: 1253.0204
Ne: 69931.4519 17 idx: 18
name: 3661
dist: 1308980.0000
support: 0
height: 2295596.0000
g: 1486.6055
Ne: 63266.9525 18 idx: 19
name: 3662
dist: 844482.0000
support: 0
height: 2760094.0000
g: 1520.5470
Ne: 90699.4793 19 idx: 20
name: 3805
dist: 763922.0000
support: 0
height: 3604576.0000
g: 1503.0850
Ne: 76586.1675 20 idx: 21
name: 3953
dist: 1737370.0000
support: 0
height: 4368498.0000
g: 1377.1422
Ne: 87397.5537 21 idx: 22
name: 3954
dist: 516070.0000
support: 0
height: 2268990.0000
g: 1756.0142
Ne: 53451.5167 22 idx: 23
name: 4109
dist: 1013160.0000
support: 0
height: 6105868.0000
g: 1335.9459
Ne: 75152.0754 23 idx: 24
name: 4110
dist: 2658110.0000
support: 0
height: 4460918.0000
g: 2370.8575
Ne: 51

**Adjust edge lengths to account for generation times in the `ipcoal` simulation:**

In [44]:
# divide the edge lengths (in abosolute time) by the generation time, to get number of generations for each edge
ttree = s_plot.set_node_values(
    "dist",
    {i.name: i.dist / (i.g/365) for i in s_plot.get_feature_dict()}
)

In [45]:
ttree.draw(ts='p',
          height=500,
          width=600);

<svg class="toyplot-canvas-Canvas" height="500.0px" id="t85732d92483840e8a60dae49de0c1f12" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 600.0 500.0" width="600.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Mustela_eversmanii_MUSTELIDAE_CARNIVORA Mustela_putorius_MUSTELIDAE_CARNIVORA Mustela_lutreola_MUSTELIDAE_CARNIVORA Mustela_sibirica_MUSTELIDAE_CARNIVORA Mustela_itatsi_MUSTELIDAE_CARNIVORA Mustela_altaica_MUSTELIDAE_CARNIVORA Mustela_nivalis_MUSTELIDAE_CARNIVORA Mustela_erminea_MUSTELIDAE_CARNIVORA Mustela_kathiah_MUSTELIDAE_CARNIVORA Mustela_nudipes_MUSTELIDAE_CARNIVORA Mustela_strigidorsa_MUSTELIDAE_CARNIVORA Mustela_felipei_MUSTELIDAE_CARNIVORA Mustela_africana_MUSTELIDAE_CARNIVORA Mustela_frenata_MUSTELIDAE_CARNIVORA Neovison_vison_MUSTELIDAE_CARNIVORA idx: 0
name: Mustela_eversmanii_MUSTELIDAE_CARNIVORA
dist: 159446.7005
support: 0
height: 316655.8073
g: 1222.3850
Ne: 76262.1310 0 idx: 1
name: Mustela_putorius_MUSTELIDAE_CARNIVORA
dist: 117948.2336
support: 0
height: 358154.2743
g: 1652.4644
Ne: 62002.2603 1 idx: 2
name: Mustela_lutreola_MUSTELIDAE_CARNIVORA
dist: 671635.0000
support: 0
height: 9966.2910
g: 730.0000
Ne: 55661.9256 2 idx: 3
name: Mustela_sibirica_MUSTELIDAE_CARNIVORA
dist: 418633.6922
support: 0
height: 379905.4477
g: 1484.7922
Ne: 82013.4602 3 idx: 4
name: Mustela_itatsi_MUSTELIDAE_CARNIVORA
dist: 433814.1552
support: 0
height: 537351.7294
g: 1931.4500
Ne: 50574.9310 4 idx: 5
name: Mustela_altaica_MUSTELIDAE_CARNIVORA
dist: 545470.3557
support: 0
height: 544370.0447
g: 1846.9000
Ne: 62140.0346 5 idx: 6
name: Mustela_nivalis_MUSTELIDAE_CARNIVORA
dist: 843606.0156
support: 0
height: 246234.3849
g: 1194.1940
Ne: 119258.9241 6 idx: 7
name: Mustela_erminea_MUSTELIDAE_CARNIVORA
dist: 1478060.4164
support: 0
height: 0.0000
g: 1078.7779
Ne: 113010.2376 7 idx: 8
name: Mustela_kathiah_MUSTELIDAE_CARNIVORA
dist: 1778585.5998
support: 0
height: 159950.1758
g: 1253.0400
Ne: 50508.5112 8 idx: 9
name: Mustela_nudipes_MUSTELIDAE_CARNIVORA
dist: 676202.8195
support: 0
height: 1129919.8017
g: 2407.9050
Ne: 52269.5290 9 idx: 10
name: Mustela_strigidorsa_MUSTELIDAE_CARNIVORA
dist: 697671.2543
support: 0
height: 1108451.3668
g: 2333.8100
Ne: 50719.7194 10 idx: 11
name: Mustela_felipei_MUSTELIDAE_CARNIVORA
dist: 427034.7648
support: 0
height: 569163.4425
g: 1939.3683
Ne: 50003.5834 11 idx: 12
name: Mustela_africana_MUSTELIDAE_CARNIVORA
dist: 526609.5024
support: 0
height: 469588.7049
g: 1572.6600
Ne: 56899.4500 12 idx: 13
name: Mustela_frenata_MUSTELIDAE_CARNIVORA
dist: 1030865.1183
support: 0
height: 72601.8990
g: 986.1105
Ne: 69498.7776 13 idx: 14
name: Neovison_vison_MUSTELIDAE_CARNIVORA
dist: 1314393.8181
support: 0
height: 415408.4700
g: 1473.6500
Ne: 70970.4741 14 idx: 15
name: 3350
dist: 205498.7832
support: 0
height: 476102.5078
g: 1437.4247
Ne: 69132.1956 15 idx: 16
name: 3426
dist: 116937.8489
support: 0
height: 681601.2910
g: 1122.7437
Ne: 63140.2669 16 idx: 17
name: 3526
dist: 172626.7447
support: 0
height: 798539.1399
g: 1253.0204
Ne: 69931.4519 17 idx: 18
name: 3661
dist: 321388.3650
support: 0
height: 971165.8846
g: 1486.6055
Ne: 63266.9525 18 idx: 19
name: 3662
dist: 202713.8491
support: 0
height: 1089840.4005
g: 1520.5470
Ne: 90699.4793 19 idx: 20
name: 3805
dist: 185506.1669
support: 0
height: 1292554.2496
g: 1503.0850
Ne: 76586.1675 20 idx: 21
name: 3953
dist: 460475.3592
support: 0
height: 1478060.4164
g: 1377.1422
Ne: 87397.5537 21 idx: 22
name: 3954
dist: 107268.8100
support: 0
height: 996198.2073
g: 1756.0142
Ne: 53451.5167 22 idx: 23
name: 4109
dist: 276810.1539
support: 0
height: 1938535.7756
g: 1335.9459
Ne: 75152.0754 23 idx: 24
name: 4110
dist: 409223.3084
suppo

**Simulate with `ipcoal`:**

In [46]:
mod = ipcoal.Model(ttree, seed=333)
mod.sim_trees(nloci=5, nsites=1e5)

**Look at the resulting dataframe:**

In [47]:
mod.df

,locus,start,end,nbps,nsnps,tidx,genealogy
0,0,0,197,197,0,0,((Neovison_vison_MUSTELI...
1,0,197,523,326,0,1,((Neovison_vison_MUSTELI...
2,0,523,959,436,0,2,((Neovison_vison_MUSTELI...
3,0,959,1460,501,0,3,((Neovison_vison_MUSTELI...
4,0,1460,1549,89,0,4,((Neovison_vison_MUSTELI...
...,...,...,...,...,...,...,...
2165,4,99445,99487,42,0,421,(((Mustela_nudipes_MUSTE...
2166,4,99487,99493,6,0,422,((Mustela_kathiah_MUSTEL...
2167,4,99493,99510,17,0,423,((Mustela_kathiah_MUSTEL...
2168,4,99510,99986,476,0,424,((Mustela_kathiah_MUSTEL...


**How long are the genealogies?**

In [48]:
canvas = toyplot.Canvas(width=300, height=300)
axes = canvas.cartesian()
bars = axes.bars(np.histogram(mod.df.nbps, 100,range=[0,1000]))

<svg class="toyplot-canvas-Canvas" height="300.0px" id="taa88de7b271546b28f5dfc42cdb0856a" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 300.0 300.0" width="300.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 500 1000 0 40 80 120

**What do the genealogies look like?**

In [49]:
# draw linked genealogies
toytree.mtree(mod.df.genealogy).draw_tree_grid(tip_labels=False);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t4fdf61db5851440fa94ed4e6e54aca2f" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 800.0 250.0" width="800.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink">

In [50]:
# draw unlinked genealogies
toytree.mtree(mod.df[mod.df.tidx==0].genealogy).draw_tree_grid(tip_labels=False);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t0ee6e0d589994bb58f080351c4c07baa" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 800.0 250.0" width="800.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink">

### Simulate on a larger clade  
  
Let's scale up to Mustelidae, the whole family containing weasels, ferrets, otters, etc.

In [51]:
# copy the whole mammal tree to prune down
s_plot = deepcopy(ntree)

In [52]:
# subset the Mustelidae
mrca = s_plot.get_mrca_idx_from_tip_labels(wildcard="MUSTELIDAE")
snode = s_plot.treenode.search_nodes(idx=mrca)[0]
s_plot.treenode.prune(snode.get_descendants())
s_plot.ntips

45

In [53]:
# scale the height from millions of years to years
s_plot = s_plot.mod.node_scale_root_height(s_plot.treenode.height*1e6)
s_plot.treenode.height

15154079.999999998

**Visualize the species tree with generation time (node colors) and Ne (edge widths) mappled on:**

In [54]:
# draw tree with reconstructed g values as node colors and Ne values as edge widths
s_plot.draw(
    ts='p', 
    #node_labels=recon_tree.get_node_values("Ne", 1, 1),
    node_colors=[colormap.colors(i, 
                                 np.min(s_plot.get_node_values("g",1,1)), 
                                 np.max(s_plot.get_node_values("g",1,1))
                                ) for i in s_plot.get_node_values('g', 1, 1)],
    height=500,
    width = 1000,
);

<svg class="toyplot-canvas-Canvas" height="500.0px" id="tc776d1edb19f4715917b9194b6f8939e" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 1000.0 500.0" width="1000.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Aonyx_cinerea_MUSTELIDAE_CARNIVORA Lutrogale_perspicillata_MUSTELIDAE_CARNIVORA Aonyx_capensis_MUSTELIDAE_CARNIVORA Lutra_lutra_MUSTELIDAE_CARNIVORA Lutra_sumatrana_MUSTELIDAE_CARNIVORA Lontra_provocax_MUSTELIDAE_CARNIVORA Lontra_longicaudis_MUSTELIDAE_CARNIVORA Lontra_canadensis_MUSTELIDAE_CARNIVORA Pteronura_brasiliensis_MUSTELIDAE_CARNIVORA Ictonyx_striatus_MUSTELIDAE_CARNIVORA Poecilogale_albinucha_MUSTELIDAE_CARNIVORA Ictonyx_libyca_MUSTELIDAE_CARNIVORA Vormela_peregusna_MUSTELIDAE_CARNIVORA Lyncodon_patagonicus_MUSTELIDAE_CARNIVORA Galictis_vittata_MUSTELIDAE_CARNIVORA Galictis_cuja_MUSTELIDAE_CARNIVORA Mustela_eversmanii_MUSTELIDAE_CARNIVORA Mustela_putorius_MUSTELIDAE_CARNIVORA Mustela_lutreola_MUSTELIDAE_CARNIVORA Mustela_sibirica_MUSTELIDAE_CARNIVORA Mustela_itatsi_MUSTELIDAE_CARNIVORA Mustela_altaica_MUSTELIDAE_CARNIVORA Mustela_nivalis_MUSTELIDAE_CARNIVORA Mustela_erminea_MUSTELIDAE_CARNIVORA Mustela_kathiah_MUSTELIDAE_CARNIVORA Mustela_nudipes_MUSTELIDAE_CARNIVORA Mustela_strigidorsa_MUSTELIDAE_CARNIVORA Mustela_felipei_MUSTELIDAE_CARNIVORA Mustela_africana_MUSTELIDAE_CARNIVORA Mustela_frenata_MUSTELIDAE_CARNIVORA Neovison_vison_MUSTELIDAE_CARNIVORA Melogale_moschata_MUSTELIDAE_CARNIVORA Melogale_personata_MUSTELIDAE_CARNIVORA Martes_martes_MUSTELIDAE_CARNIVORA Martes_zibellina_MUSTELIDAE_CARNIVORA Martes_melampus_MUSTELIDAE_CARNIVORA Martes_americana_MUSTELIDAE_CARNIVORA Martes_foina_MUSTELIDAE_CARNIVORA Martes_flavigula_MUSTELIDAE_CARNIVORA Gulo_gulo_MUSTELIDAE_CARNIVORA Martes_pennanti_MUSTELIDAE_CARNIVORA Eira_barbara_MUSTELIDAE_CARNIVORA Mellivora_capensis_MUSTELIDAE_CARNIVORA Arctonyx_collaris_MUSTELIDAE_CARNIVORA Taxidea_taxus_MUSTELIDAE_CARNIVORA idx: 0
name: Aonyx_cinerea_MUSTELIDAE_CARNIVORA
dist: 2437390.0000
support: 0
height: 47.0000
g: 3650.0000
Ne: 55827.5585 0 idx: 1
name: Lutrogale_perspicillata_MUSTELIDAE_CARNIVORA
dist: 2437390.0000
support: 0
height: 47.0000
g: 3650.0000
Ne: 58736.1764 1 idx: 2
name: Aonyx_capensis_MUSTELIDAE_CARNIVORA
dist: 3210040.0000
support: 0
height: 42.0000
g: 1622.0935
Ne: 72467.6876 2 idx: 3
name: Lutra_lutra_MUSTELIDAE_CARNIVORA
dist: 2053010.0000
support: 0
height: 42.0000
g: 2772.9621
Ne: 91052.9944 3 idx: 4
name: Lutra_sumatrana_MUSTELIDAE_CARNIVORA
dist: 2053010.0000
support: 0
height: 42.0000
g: 3650.0000
Ne: 52114.8261 4 idx: 5
name: Lontra_provocax_MUSTELIDAE_CARNIVORA
dist: 1108700.0000
support: 0
height: 42.0000
g: 3650.0000
Ne: 51276.7548 5 idx: 6
name: Lontra_longicaudis_MUSTELIDAE_CARNIVORA
dist: 1108700.0000
support: 0
height: 42.0000
g: 3442.5583
Ne: 70442.5519 6 idx: 7
name: Lontra_canadensis_MUSTELIDAE_CARNIVORA
dist: 3445000.0000
support: 0
height: 42.0000
g: 3235.1167
Ne: 68858.0191 7 idx: 8
name: Pteronura_brasiliensis_MUSTELIDAE_CARNIVORA
dist: 8883550.0000
support: 0
height: 48.0000
g: 2190.0000
Ne: 62811.4751 8 idx: 9
name: Ictonyx_striatus_MUSTELIDAE_CARNIVORA
dist: 3864740.0000
support: 0
height: 50.0000
g: 1650.8092
Ne: 76475.5926 9 idx: 10
name: Poecilogale_albinucha_MUSTELIDAE_CARNIVORA
dist: 3864740.0000
support: 0
height: 50.0000
g: 1241.5557
Ne: 60350.9422 10 idx: 11
name: Ictonyx_libyca_MUSTELIDAE_CARNIVORA
dist: 5446870.0000
support: 0
height: 40.0000
g: 792.6600
Ne: 69490.5436 11 idx: 12
name: Vormela_peregusna_MUSTELIDAE_CARNIVORA
dist: 8222440.0000
support: 0
height: 40.0000
g: 1120.7628
Ne: 64093.3045 12 idx: 13
name: Lyncodon_patagonicus_MUSTELIDAE_CARNIVORA
dist: 2757640.0000
support: 0
heigh

**Adjust edge lengths to account for generation times in the `ipcoal` simulation:**

In [55]:
ttree = s_plot.set_node_values(
    "dist",
    {i.name: i.dist / (i.g/365) for i in s_plot.get_feature_dict()}
)

In [56]:
ttree.draw(ts='p',
          height=500,
          width=1000);

<svg class="toyplot-canvas-Canvas" height="500.0px" id="tc465388a169b4937bcbebe6ac4e8c176" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 1000.0 500.0" width="1000.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> Aonyx_cinerea_MUSTELIDAE_CARNIVORA Lutrogale_perspicillata_MUSTELIDAE_CARNIVORA Aonyx_capensis_MUSTELIDAE_CARNIVORA Lutra_lutra_MUSTELIDAE_CARNIVORA Lutra_sumatrana_MUSTELIDAE_CARNIVORA Lontra_provocax_MUSTELIDAE_CARNIVORA Lontra_longicaudis_MUSTELIDAE_CARNIVORA Lontra_canadensis_MUSTELIDAE_CARNIVORA Pteronura_brasiliensis_MUSTELIDAE_CARNIVORA Ictonyx_striatus_MUSTELIDAE_CARNIVORA Poecilogale_albinucha_MUSTELIDAE_CARNIVORA Ictonyx_libyca_MUSTELIDAE_CARNIVORA Vormela_peregusna_MUSTELIDAE_CARNIVORA Lyncodon_patagonicus_MUSTELIDAE_CARNIVORA Galictis_vittata_MUSTELIDAE_CARNIVORA Galictis_cuja_MUSTELIDAE_CARNIVORA Mustela_eversmanii_MUSTELIDAE_CARNIVORA Mustela_putorius_MUSTELIDAE_CARNIVORA Mustela_lutreola_MUSTELIDAE_CARNIVORA Mustela_sibirica_MUSTELIDAE_CARNIVORA Mustela_itatsi_MUSTELIDAE_CARNIVORA Mustela_altaica_MUSTELIDAE_CARNIVORA Mustela_nivalis_MUSTELIDAE_CARNIVORA Mustela_erminea_MUSTELIDAE_CARNIVORA Mustela_kathiah_MUSTELIDAE_CARNIVORA Mustela_nudipes_MUSTELIDAE_CARNIVORA Mustela_strigidorsa_MUSTELIDAE_CARNIVORA Mustela_felipei_MUSTELIDAE_CARNIVORA Mustela_africana_MUSTELIDAE_CARNIVORA Mustela_frenata_MUSTELIDAE_CARNIVORA Neovison_vison_MUSTELIDAE_CARNIVORA Melogale_moschata_MUSTELIDAE_CARNIVORA Melogale_personata_MUSTELIDAE_CARNIVORA Martes_martes_MUSTELIDAE_CARNIVORA Martes_zibellina_MUSTELIDAE_CARNIVORA Martes_melampus_MUSTELIDAE_CARNIVORA Martes_americana_MUSTELIDAE_CARNIVORA Martes_foina_MUSTELIDAE_CARNIVORA Martes_flavigula_MUSTELIDAE_CARNIVORA Gulo_gulo_MUSTELIDAE_CARNIVORA Martes_pennanti_MUSTELIDAE_CARNIVORA Eira_barbara_MUSTELIDAE_CARNIVORA Mellivora_capensis_MUSTELIDAE_CARNIVORA Arctonyx_collaris_MUSTELIDAE_CARNIVORA Taxidea_taxus_MUSTELIDAE_CARNIVORA idx: 0
name: Aonyx_cinerea_MUSTELIDAE_CARNIVORA
dist: 243739.0000
support: 0
height: 2897912.8676
g: 3650.0000
Ne: 55827.5585 0 idx: 1
name: Lutrogale_perspicillata_MUSTELIDAE_CARNIVORA
dist: 243739.0000
support: 0
height: 2897912.8676
g: 3650.0000
Ne: 58736.1764 1 idx: 2
name: Aonyx_capensis_MUSTELIDAE_CARNIVORA
dist: 722316.3077
support: 0
height: 2496600.0599
g: 1622.0935
Ne: 72467.6876 2 idx: 3
name: Lutra_lutra_MUSTELIDAE_CARNIVORA
dist: 270234.0052
support: 0
height: 2832391.8398
g: 2772.9621
Ne: 91052.9944 3 idx: 4
name: Lutra_sumatrana_MUSTELIDAE_CARNIVORA
dist: 205301.0000
support: 0
height: 2897324.8450
g: 3650.0000
Ne: 52114.8261 4 idx: 5
name: Lontra_provocax_MUSTELIDAE_CARNIVORA
dist: 110870.0000
support: 0
height: 2968025.0214
g: 3650.0000
Ne: 51276.7548 5 idx: 6
name: Lontra_longicaudis_MUSTELIDAE_CARNIVORA
dist: 117550.8040
support: 0
height: 2961344.2173
g: 3442.5583
Ne: 70442.5519 6 idx: 7
name: Lontra_canadensis_MUSTELIDAE_CARNIVORA
dist: 388679.9548
support: 0
height: 2930678.2281
g: 3235.1167
Ne: 68858.0191 7 idx: 8
name: Pteronura_brasiliensis_MUSTELIDAE_CARNIVORA
dist: 1480591.6667
support: 0
height: 2427160.6820
g: 2190.0000
Ne: 62811.4751 8 idx: 9
name: Ictonyx_striatus_MUSTELIDAE_CARNIVORA
dist: 854508.2851
support: 0
height: 1254329.2938
g: 1650.8092
Ne: 76475.5926 9 idx: 10
name: Poecilogale_albinucha_MUSTELIDAE_CARNIVORA
dist: 1136179.5180
support: 0
height: 972658.0609
g: 1241.5557
Ne: 60350.9422 10 idx: 11
name: Ictonyx_libyca_MUSTELIDAE_CARNIVORA
dist: 2508146.6833
support: 0
height: 0.0000
g: 792.6600
Ne: 69490.5436 11 idx: 12
name: Vormela_peregusna_MUSTELIDAE_CARNIVORA
dist: 2677810.7365
support: 0
height: 681750.4891
g: 1120.7628
Ne: 64093.3045 12 idx: 13
name: Lyncodon_patagonicus_MUSTELID

**Simulate with `ipcoal`:**

In [57]:
mod = ipcoal.Model(ttree, seed=333)
mod.sim_trees(nloci=5, nsites=1e5)

**Look at the resulting dataframe:**

In [58]:
mod.df

,locus,start,end,nbps,nsnps,tidx,genealogy
0,0,0,147,147,0,0,(Arctonyx_collaris_MUSTE...
1,0,147,148,1,0,1,(Arctonyx_collaris_MUSTE...
2,0,148,162,14,0,2,(Arctonyx_collaris_MUSTE...
3,0,162,359,197,0,3,(Arctonyx_collaris_MUSTE...
4,0,359,472,113,0,4,(Arctonyx_collaris_MUSTE...
...,...,...,...,...,...,...,...
6188,4,99861,99880,19,0,1216,(Taxidea_taxus_MUSTELIDA...
6189,4,99880,99884,4,0,1217,(Taxidea_taxus_MUSTELIDA...
6190,4,99884,99906,22,0,1218,(Taxidea_taxus_MUSTELIDA...
6191,4,99906,99958,52,0,1219,(Taxidea_taxus_MUSTELIDA...


**How long are the genealogies?**

In [59]:
canvas = toyplot.Canvas(width=300, height=300)
axes = canvas.cartesian()
bars = axes.bars(np.histogram(mod.df.nbps, 100,range=[0,500]))

<svg class="toyplot-canvas-Canvas" height="300.0px" id="t6dc26831562a4da285a4f11b175bf4a9" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 300.0 300.0" width="300.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 100 200 300 400 500 0 100 200 300 400

**What do the genealogies look like?**

In [60]:
# draw linked genealogies
toytree.mtree(mod.df.genealogy).draw_tree_grid(tip_labels=False);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t353d20fd2b7e4848b89be35dd2fb0046" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 800.0 250.0" width="800.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> </circl

In [61]:
# draw unlinked genealogies
toytree.mtree(mod.df[mod.df.tidx==0].genealogy).draw_tree_grid(tip_labels=False);

<svg class="toyplot-canvas-Canvas" height="250.0px" id="t5dc3618a22b6470089b0b36fe1a0e2b8" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 800.0 250.0" width="800.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <g class="toyplot-Datum" style="fill:rgb(40%,76.1%,64.7%);fill-opacity:1.0;opacity:1.0;str